In [1]:
import requests
import xmltodict
import datetime
import os
import pandas as pd

In [2]:
# uses the xmltodict library to convert XML to JSON

def convertXMLtoJSON(xml, xml_attribs=True):
    return xmltodict.parse(xml, xml_attribs=xml_attribs)

In [3]:
# Return a JSON response to an XML query defined in http://123.57.215.15:13000/CommonSearchService.svc?singleWsdl

def get(xml):
    soapaction = xml.split('<s:Body><')[1].split(' ')[0]
    headers = {'Content-Type': 'text/xml', 
               'Referer': 'http://gz.hcstzz.com', 
               'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_3) AppleWebKit/602.4.8 (KHTML, like Gecko) Version/10.0.3 Safari/602.4.8', 
               'Soapaction': "http://tempuri.org/ICommonSearchService/" + soapaction
              }
    xml = xml.encode(encoding='utf-8')
    resp = requests.post('http://123.57.215.15:13000/CommonSearchService.svc', data=xml, headers=headers).text
    return convertXMLtoJSON(resp)

# Reports

In [4]:
### Get all data

# To edit the xml query:
#  - change 'SelectAllReportListByDate' to another xs:element defined in the 
#    CommonSearchService.svc file linked above.
#  - replace the tags within <SelectAllReportListByDate> with the elements 
#    described in the <xs:sequence> tag in the CommonSearchService.svc file. 

xml_query = """<s:Envelope xmlns:s="http://schemas.xmlsoap.org/soap/envelope/"><s:Body><SelectAllReportListByDate xmlns="http://tempuri.org/"><ip>hcstzz.com</ip><province></province><city></city><year></year><quarter></quarter><status></status><startDate></startDate><endDate></endDate></SelectAllReportListByDate></s:Body></s:Envelope>"""
response = get(xml_query)
response = response['s:Envelope']['s:Body']['SelectAllReportListByDateResponse']['SelectAllReportListByDateResult']['a:M_T_Report']
response = pd.DataFrame(response)
response.shape

(8667, 57)

In [5]:
# Columns to extract from the full dataframe

extractCols = ['a:Report_ID', 'a:Qwyc', 'a:Ljpf', 'a:Ysyc', 'a:Wslp', 'a:River_Name', 
               'a:Report_Content1', 'a:Report_Content2', 
               'a:Report_Latitude', 'a:Report_Longitude', 'a:Report_SubTime', 'a:adcode',
               'a:Custom_NickName',
               'a:ReportQuarter', 'a:ReportYear', 
               'a:AreaName', 'a:AreaCode', 'a:city', 'a:district', 'a:province']

renameCols = {
    'a:Qwyc': 'smell', 
    'a:Ljpf': 'isTrashFloating', 
    'a:Ysyc': 'isWaterColorNormal', 
    'a:Wslp': 'sewage'  
}

In [6]:
# Define path to location for file export. 

ROOT = os.pardir
RAW_DIR    = os.path.join(ROOT, 'data', 'raw', 'smelly_water')
OUT_PATH = os.path.join(RAW_DIR, "all_smelly.xls")

In [7]:
# Export response

response = response[extractCols].rename(columns=renameCols)
response.to_excel(OUT_PATH)

In [8]:
# Check what quarters are present in each year. 

print(response[response['a:ReportYear'] == '2016']['a:ReportQuarter'].unique())
print(response[response['a:ReportYear'] == '2017']['a:ReportQuarter'].unique())

['4' '3' '2' '1']
['4' '3' '2' '1']


# Administrative Data

In [14]:
# all

xml_query = """<s:Envelope xmlns:s="http://schemas.xmlsoap.org/soap/envelope/"><s:Body><SelectWaterDeclareBaseInfoByWhereSql xmlns="http://tempuri.org/"><whereSql></whereSql></SelectWaterDeclareBaseInfoByWhereSql></s:Body></s:Envelope>"""
response = get(xml_query)
response = response['s:Envelope']['s:Body']['SelectWaterDeclareBaseInfoByWhereSqlResponse']['SelectWaterDeclareBaseInfoByWhereSqlResult']['a:M_T_Water_Declare_based_info']
response = pd.DataFrame(response)
response = response.rename(columns=renameCols)
response.to_excel(os.path.join(RAW_DIR, 'all_smelly_water_admin_data.xls'))
response.shape

In [12]:
# wei3 qi3dong4
# not started

xml_query = """<s:Envelope xmlns:s="http://schemas.xmlsoap.org/soap/envelope/"><s:Body><SelectWaterDeclareBaseInfoByWhereSql xmlns="http://tempuri.org/"><whereSql> and States ='未启动' </whereSql></SelectWaterDeclareBaseInfoByWhereSql></s:Body></s:Envelope>"""
response = get(xml_query)
response = response['s:Envelope']['s:Body']['SelectWaterDeclareBaseInfoByWhereSqlResponse']['SelectWaterDeclareBaseInfoByWhereSqlResult'] #['a:M_T_Water_Declare_based_info']
response = pd.DataFrame(response)
response = response.rename(columns=renameCols)
response.to_excel(os.path.join(RAW_DIR, 'wei_qidong_smelly_water_admin.xls'))
response.shape

In [16]:
# fang1an4 zhi4ding4
# ___ development

xml_query = """<s:Envelope xmlns:s="http://schemas.xmlsoap.org/soap/envelope/"><s:Body><SelectWaterDeclareBaseInfoByWhereSql xmlns="http://tempuri.org/"><whereSql> and States ='方案制定' </whereSql></SelectWaterDeclareBaseInfoByWhereSql></s:Body></s:Envelope>"""
response = get(xml_query)
response = response['s:Envelope']['s:Body']['SelectWaterDeclareBaseInfoByWhereSqlResponse']['SelectWaterDeclareBaseInfoByWhereSqlResult']['a:M_T_Water_Declare_based_info']
response = pd.DataFrame(response)
response = response.rename(columns=renameCols)
response.to_excel(os.path.join(RAW_DIR, 'fangan_zhiding_smelly_water_admin.xls'))
response.shape

(91, 64)

In [17]:
# zhi4li3 zhong1
# governance

xml_query = """<s:Envelope xmlns:s="http://schemas.xmlsoap.org/soap/envelope/"><s:Body><SelectWaterDeclareBaseInfoByWhereSql xmlns="http://tempuri.org/"><whereSql> and States ='治理中' </whereSql></SelectWaterDeclareBaseInfoByWhereSql></s:Body></s:Envelope>"""
response = get(xml_query)
response = response['s:Envelope']['s:Body']['SelectWaterDeclareBaseInfoByWhereSqlResponse']['SelectWaterDeclareBaseInfoByWhereSqlResult']['a:M_T_Water_Declare_based_info']
response = pd.DataFrame(response)
response = response.rename(columns=renameCols)
response.to_excel(os.path.join(RAW_DIR, 'zhili_zhong_smelly_water_admin.xls'))
response.shape

(264, 64)

In [18]:
# wan2cheng2 zhi4li3
# completed governance?

xml_query = """<s:Envelope xmlns:s="http://schemas.xmlsoap.org/soap/envelope/"><s:Body><SelectWaterDeclareBaseInfoByWhereSql xmlns="http://tempuri.org/"><whereSql> and States ='完成治理' </whereSql></SelectWaterDeclareBaseInfoByWhereSql></s:Body></s:Envelope>"""
response = get(xml_query)
response = response['s:Envelope']['s:Body']['SelectWaterDeclareBaseInfoByWhereSqlResponse']['SelectWaterDeclareBaseInfoByWhereSqlResult']['a:M_T_Water_Declare_based_info']
response = pd.DataFrame(response)
response = response.rename(columns=renameCols)
response.to_excel(os.path.join(RAW_DIR, 'wancheng_zhili_smelly_water_admin.xls'))
response.shape

(1745, 64)

In [11]:
# The number of actions in the different categories is equal to the number of actions under 'all'.
794+656+642+8 == 2100

True